# Dataset Benchmark

In [39]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from numpy import linalg as LA 

from sgnet import *
from trainer import *
from gazecapture_sequence import *

# Config to Context

In [79]:
config = {
    'num_epochs' : 20,
    'num_workers' : 8,
    'batch_size' : 32,
    'resource_path': '../../../../data-archive/faze-resources/',
    'npz_root_path': '../../../../data-archive/faze-recode-profile-npz/',
}

ctx = Context.create(config)

# Model & Dataset

In [ ]:
def benchmark(dataset, num_epochs=2):
    import time
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # 훈련 스텝마다 실행
            time.sleep(0.01)
    tf.print("실행 시간:", time.perf_counter() - start_time)

In [ ]:
train_seq = GazeCaptureNpzSequence.create(ctx, 'train')
valid_seq = GazeCaptureNpzSequence.create(ctx, 'val')

In [ ]:
def get_generator(dataset):
    def generator():
        it = iter(dataset)
        while True:
            inputs, targets = next(it)
            inputs = (inputs['left_eye_patch'], 
                      inputs['right_eye_patch'],  
                      inputs['eye_corner_landmark'])   # WARN: tuple로 지정해야 함 
            yield inputs, targets
    return generator

In [ ]:
n = ctx.batch_size
from_gen = tf.data.Dataset.from_generator
outputs_shapes = (((n,64,64,3), (n,64,64,3), (n,8)), (n,2))
outputs_types = ((tf.float32, tf.float32, tf.float32), tf.float32)

ds = from_gen(get_generator(valid_seq), output_types=outputs_types, output_shapes=outputs_shapes)

In [ ]:
# Prefetch 비교
# ds = ds.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# prefetch 실행 시간: 918.9525989890099
benchmark(ds, num_epochs=1)

In [ ]:
benchmark(ds, num_epochs=1)

In [89]:
y_t = np.random.uniform(0, 6.0, [10, 2])
y_h = np.random.uniform(0, 6.0, [10, 2])

In [90]:
print(y_t)
print(y_h)

[[5.54312244 1.46991857]
 [1.25319167 5.15543502]
 [4.75982232 2.71381462]
 [5.07445951 3.11153309]
 [3.78341594 5.37046205]
 [0.29686938 3.76341711]
 [5.72138805 1.59467643]
 [4.56022154 4.22253876]
 [4.54514429 3.70970911]
 [4.32339819 5.86130248]]
[[5.92686369 5.2258164 ]
 [2.26762047 2.15865895]
 [3.45627428 2.32404691]
 [0.74661035 3.18521119]
 [2.29044429 5.31593046]
 [2.70956536 0.73459192]
 [1.80178833 2.15170688]
 [5.19357775 1.0005738 ]
 [1.45815099 2.04846908]
 [4.46715947 0.80068484]]


In [91]:
mean_squared_error(y_t, y_h)

6.322647550747376

In [92]:
mse = (np.square(y_h - y_t)).mean()
print(mse)

6.322647550747376


In [93]:
mean_absolute_error(y_t, y_h)

1.9759637483548413

In [94]:
mae = (y_h - y_t)
print(mae)

[[ 0.38374125  3.75589782]
 [ 1.0144288  -2.99677607]
 [-1.30354804 -0.38976772]
 [-4.32784915  0.0736781 ]
 [-1.49297165 -0.05453158]
 [ 2.41269598 -3.02882519]
 [-3.91959972  0.55703045]
 [ 0.63335622 -3.22196496]
 [-3.0869933  -1.66124003]
 [ 0.14376129 -5.06061764]]


In [95]:
norms = LA.norm(y_h - y_t, axis=1)
print(norms)

[3.77545041 3.16381615 1.36057207 4.32847626 1.49396722 3.87232281
 3.95898281 3.28362579 3.5056021  5.06265921]


In [96]:
np.sum(norms) / 10

3.380547482033916

51200000
22164480
